## LeanSpeech Training: HFC-Female (en-US)
This notebook allows you to train [LeanSpeech TTS](https://github.com/mush42/leanspeech) on [HiFiCaptin en-US female dataset](https://ast-astrec.nict.go.jp/en/release/hi-fi-captain/)


## Plumming

In [ ]:
#@markdown ### Google Colab Anti-Disconnect
#@markdown Avoid automatic disconnection. Still, it will disconnect after **6 to 12 hours**.

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))


#@markdown ### Check GPU type
#@markdown A higher capable GPU can lead to faster training speeds. By default, you will have a **Tesla T4**.
!nvidia-smi

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## Prepare environment

In [ ]:
#@markdown ### Clone leanSpeech repository

import os

if not os.path.isdir(os.path.join(os.getcwd(), "leanspeech")):
    print("Cloning leanspeech repository...")
    !git clone --depth=1 https://github.com/mush42/leanspeech

#@markdown ### Upgrade packages

!pip3 install --upgrade pip setuptools wheel

#@markdown ### Install leanspeech dependencies

%cd /content/leanspeech
!pip3 install -r requirements.txt

# Install onnxruntime with supported cua version
!pip3 install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/

## Upload sentences to generate synthetic dataset

In [ ]:
from google.colab import files

uploaded = files.upload()
(filename, file_content) = tuple(uploaded.items())[0]
print(f"Uploaded file: {filename}")
source_text = file_content.decode("utf-8")
source_lines = source_text.splitlines()
source_lines = [line for l in source_lines if (line := l.strip())]
print(f"Found {len(source_lines)} lines in the uploaded file")
with open("sentences.txt", "w", encoding="utf-8") as outfile:
    outfile.write("\n".join(source_lines ))
print("Wrote lines to `sentences.txt`")

## Generate synthetic dataset

In [ ]:
print("Downloading Matcha ONNX...")
%cd /content/leanspeech
!wget -O "matcha-hfc_female.onnx" "https://drive.google.com/uc?export=download&id=18ysTCnXipCzzK6LxJtXFJvpDa6QKt7Ol"

%cd /content/leanspeech
!python3 -m leanspeech.tools.matcha_synthetic \
    --batch-size 64 \
    --mel-mean "-6.38385" \
    --mel-std 2.541796 \
    --val-percent 0.025 \
    --cuda \
    matcha-hfc_female.onnx \
    en-us \
    /content/sentences.txt \
    ./data/synthetic


## Start training

In [ ]:
%cd /content/leanspeech
!python3 -m leanspeech.train \
    experiment="hfc_female-en_US" \
    data.train_filelist_path="data/synthetic/train.txt" \
    data.valid_filelist_path="data/synthetic/val.txt" \
    data.batch_size=64 \
    paths.log_dir="/content/drive/MyDrive/leanspeech/logs"
